In [2]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10

/sdc1/caiyunuo/anaconda3/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from glob import glob
file_list = glob("./dataset/train/*/*.JPEG")

In [4]:
len(file_list)

1281167

In [8]:
from PIL import Image
import numpy as np

idx = 0
image_path = file_list[idx]

image = Image.open(image_path)
img = np.array(image)

In [ ]:
from tqdm import tqdm
from PIL import Image
import numpy as np

max_shape0 = 0
max_shape1 = 0
for idx in tqdm(range(len(file_list))):    
    image_path = file_list[idx]
    image = Image.open(image_path)
    img = np.array(image)
    if img.shape[-1] != 3:
        print(file_list[idx])

In [13]:
print(max_shape0)
print(max_shape1)

6530
9331


In [14]:
from torchvision.models import resnet18

encoder = resnet18()

In [15]:
from torchvision import models

models = models.__dict__

<function resnet18 at 0x7f19a7792ee0>


In [11]:
from dataset import ImageNetDatasetForMoCoPretraining
from torch.utils.data import DataLoader

dataset = ImageNetDatasetForMoCoPretraining()
dataloader = DataLoader(dataset, batch_size=32, num_workers=32)

for i, data in enumerate(dataloader):
    print(data[1])
    print(data[0].shape)
    break

tensor([[[[-0.2342, -2.0494, -2.1008,  ...,  0.3652,  0.4166,  0.6563],
          [ 0.2282, -1.0390, -1.8782,  ...,  0.5536,  0.6049,  0.8618],
          [ 0.6392, -0.0801, -1.6727,  ...,  0.8618,  0.8447,  0.8276],
          ...,
          [-1.1760, -1.3302, -1.5357,  ...,  0.4679,  0.5536,  0.7077],
          [-0.1999, -0.4054, -0.7993,  ...,  0.3994,  0.4851,  0.4166],
          [-0.8849, -1.0390, -1.3130,  ...,  0.2796,  0.3481,  0.1254]],

         [[-0.2325, -1.8957, -2.0357,  ..., -0.6352, -0.5126, -0.2675],
          [ 0.2227, -1.0203, -2.0182,  ..., -0.3375, -0.2325,  0.1176],
          [ 0.5553, -0.3025, -1.8782,  ..., -0.0049,  0.0826,  0.2052],
          ...,
          [-1.0028, -1.1604, -1.4230,  ...,  1.0980,  1.1856,  1.3431],
          [-0.0049, -0.2150, -0.6001,  ...,  1.0105,  1.0980,  1.0280],
          [-0.7052, -0.8627, -1.0728,  ...,  0.8880,  0.9580,  0.7479]],

         [[ 0.4614, -1.3339, -1.7696,  ...,  1.4025,  1.4374,  1.6291],
          [ 0.9842, -0.3927, -

In [16]:
for i in range(100):
    print(i)
    print(i, end='\r')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [38]:
from torchvision.datasets import CIFAR10
import torchvision

trainset = CIFAR10(root="/sdc1/caiyunuo/pretraining-ResNet/dataset/CIFAR10", train=False, transform=torchvision.transforms.ToTensor())

In [39]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(trainset, batch_size=32, shuffle=True)

In [40]:
import torch
from tqdm import tqdm
image_all = torch.zeros((32, 3, 32, 32))
for i, image in enumerate(tqdm(train_dataloader)):
    if i == 0:
        image_all = image[0]
    else:
        image_all = torch.cat((image_all, image[0]), dim=0)

100%|██████████| 313/313 [00:20<00:00, 15.31it/s]


In [41]:
temp = image_all.transpose(0, 1).reshape(3, -1)
print(temp.shape)
print(temp.mean(axis=1))
print(temp.std(axis=1, unbiased=False))

torch.Size([3, 10240000])
tensor([0.4942, 0.4851, 0.4504])
tensor([0.2467, 0.2429, 0.2616])


In [10]:
import torch
model_info = torch.load("/sdc1/caiyunuo/pretraining-ResNet/checkpoint/checkpoint-pretrained/checkpoint_0159.pth.tar")
weights = model_info['state_dict']

weights_dict = {}
for k, v in weights.items():
    new_k = k.replace('module.', '') if 'module' in k else k
    weights_dict[new_k] = v


In [13]:
from torchvision.models import resnet18
from models import MoCoRes18

model = resnet18(num_classes=128)
pretrained_model = MoCoRes18()
pretrained_model.load_state_dict(weights_dict)

<All keys matched successfully>

In [14]:
model.load_state_dict(pretrained_model.encoder_q.state_dict())

<All keys matched successfully>

In [20]:
model.fc = torch.nn.Linear(in_features=512, out_features=10)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [21]:
test_in = torch.randn(6, 3, 32, 32)
out = model(test_in)
print(out.shape)

torch.Size([6, 10])
